# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 라이브러리 불러오기

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb
import os

print("Version Pandas", pd.__version__)
print("Version Matplotlib", matplotlib.__version__)
print("Version Numpy", np.__version__)
print("Version Seaborn", sb.__version__)

# 경로 잡기
# 파일 안에 있는 데이터를 list로 반환
os.listdir('../input/tabular-playground-series-apr-2021/')


In [ ]:

BASE_DIR = '../input/tabular-playground-series-apr-2021/'
train = pd.read_csv(BASE_DIR + "train.csv")
test = pd.read_csv(BASE_DIR + "test.csv")
sample_submission = pd.read_csv(BASE_DIR + "sample_submission.csv")
train.shape, test.shape, sample_submission.shape

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
sample_submission.head()

- 데이터 합

In [ ]:
# 데이터를 상하로 합
all_df = pd.concat([train,test])
all_df.shape

- 결측치 합 % 처리
- 여기가 핵심

In [ ]:
# Start
print("Before Handling:", all_df.shape)

# Age
age_dict = all_df[['Age', 'Pclass']].dropna().groupby('Pclass').mean().round(0).to_dict()
print("Avg. Mean of Age by Pclass:", age_dict)
all_df['Age'] = all_df['Age'].fillna(all_df.Pclass.map(age_dict['Age']))

# Cabin
all_df["Cabin"].fillna("No Cabin", inplace = True)
print("Values from Cabin: ", all_df["Cabin"].unique())
all_df['Cabin_Code'] = all_df['Cabin'].fillna('X').map(lambda x: x[0].strip())
print("Values from Cabin Code: ", all_df["Cabin_Code"].unique())

# Fare
print("Avg. Mean:", np.round(all_df['Fare'].mean(), 2))
all_df['Fare'] = all_df['Fare'].fillna(round(all_df['Fare'].mean(), 2))

# Embarked
all_df["Embarked"].fillna("X", inplace = True)
print("Values from Embarked: ", all_df["Embarked"].unique())

# Delete Columns
all_df.drop(['Ticket', 'Cabin', 'Name', 'PassengerId'], axis=1, inplace=True)
print("After Handling:", all_df.shape)


- 피처 인코딩 : 문자열을 숫자로 변환
- Feature Encoding : why? - 머신러닝 = 수식이기 때문에
ex) Ordinal Encoding, Label Encoding, One-Hot Encoding

In [ ]:
cat_cols = ['Pclass', 'Sex', 'Cabin_Code', 'Embarked']
num_cols = ['Age', 'SibSp', 'Parch', 'Fare', 'Survived']

onehot_df = pd.get_dummies(all_df[cat_cols])
print("onehot_df Shape:", onehot_df.shape)

num_df = all_df[num_cols]
print("num_df Shape:", num_df.shape)

all_cleansed_df = pd.concat([num_df, onehot_df], axis=1)
print("all_cleansed_df Shape:", all_df.shape)


In [ ]:
all_cleansed_df.head()

# 머신러닝

- 데이터셋 분리
- x : 독립변수 , y : 종속변수
- test_data는 마지막 제출전에 한번

In [ ]:
X = all_cleansed_df[:train.shape[0]]
print("X Shape is:", X.shape)
y = X['Survived']
X.drop(['Survived'], axis=1, inplace=True)
test_data = all_cleansed_df[train.shape[0]:].drop(columns=['Survived'])
test_data.info()

In [ ]:
X.shape, y.shape

- 라이브러리 설치

In [ ]:
!pip install scikit-learn==0.23.2

In [ ]:
import sklearn
print(sklearn.__version__)

- 충화추출(Stratfied Sampling)
- random_state : 실험을 실행 할때마다 데이터셋이 계속 섞이면 안되니까 고정

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, stratify = X[['Pclass']], random_state=42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

- Base Model : 가장 심플한 모델
- 평가지표 
    + 분류모형(정확도_

In [ ]:
from sklearn.metrics import accuracy_score
def acc_score(y_true, y_pred, **kwargs):
    return accuracy_score(y_true, (y_pred > 0.5).astype(int), **kwargs)


In [ ]:
from sklearn import tree
# 모형
from sklearn.tree import DecisionTreeClassifier
# 모형 평가
from sklearn.metrics import roc_curve, roc_auc_score
# 시각화
from matplotlib import pyplot as plt

# 모형 만든다.
tree_model = DecisionTreeClassifier(max_depth=3)
tree_model.fit(X_train, y_train)

# 예측
predictions = tree_model.predict_proba(X_val)

# 실제와 예측 비교
AUC = roc_auc_score(y_val, predictions[:,1])
ACC = acc_score(y_val, predictions[:,1])
# 정확성
print("Model AUC:", AUC)
# 신뢰성
print("Model Accurarcy:", ACC)
print("\n")


# 시각화 하기
fpr, tpr, _ = roc_curve(y_val, predictions[:,1])

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(fpr, tpr)
ax.text(x = 0.3, 
        y = 0.4, 
        s = "Model AUC is {}\n\nModel Accuracy is {}".format(np.round(AUC, 2), np.round(ACC, 2)), 
        fontsize=16, bbox=dict(facecolor='gray', alpha=0.3))
ax.set_xlabel('FPR')
ax.set_ylabel('TPR')
ax.set_title('ROC curve')

plt.show()

## Helper class 생성

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, confusion_matrix
from matplotlib import pyplot as plt

SEED = 0 # for Reproducibility

# class 
class sk_helper(object):
    def __init__(self, model, seed = 0, params={}):
        params['random_state'] = seed
        self.model = model(**params)
        self.model_name = str(model).split(".")[-1][:-2]
        
    # train
    def train(self, X_train, y_train):
        self.model.fit(X_train, y_train)
        
    # predict
    def predict(self, y_val):
        return self.model.predict(y_val)
    
    # inner fit
    def fit(self, x, y):
        return self.model.fit(x, y)
    
    # feature importance
    def feature_importances(self, X_train, y_train):
        return self.model.fit(X_train, y_train).feature_importances_
        
    # roc_curve
    def roc_curve_graph(self, X_train, y_train, X_val, y_val):
        self.model.fit(X_train, y_train)
        
        print("model_name:", self.model_name)
        model_name = self.model_name
        preds_proba = self.model.predict_proba(X_val)
        preds = (preds_proba[:, 1] > 0.5).astype(int)
        auc = roc_auc_score(y_val, preds_proba[:, 1])
        acc = accuracy_score(y_val, preds)
        confusion = confusion_matrix(y_val, preds)
        print('Confusion Matrix')
        print(confusion)
        print("Model AUC: {0:.3f}, Model Accuracy: {1:.3f}\n".format(auc, acc))
        fpr, tpr, _ = roc_curve(y_val, predictions[:,1])
        fig, ax = plt.subplots(figsize=(10, 6))

        ax.plot(fpr, tpr)
        ax.text(x = 0.3, 
                y = 0.4, 
                s = "Model AUC is {}\n\nModel Accuracy is {}".format(np.round(auc, 2), np.round(acc, 2)), 
                fontsize=16, bbox=dict(facecolor='gray', alpha=0.3))
        ax.set_xlabel('FPR')
        ax.set_ylabel('TPR')
        ax.set_title('ROC curve of {}'.format(model_name), fontsize=16)

        plt.show()


In [ ]:
%%time
tree_params = {'max_depth': 6}

tree_model = sk_helper(model=DecisionTreeClassifier, seed=SEED, params=tree_params)
tree_model.roc_curve_graph(X_train, y_train, X_val, y_val)

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier

rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 1
}

rf_model = sk_helper(model=RandomForestClassifier, seed=SEED, params=rf_params)
rf_model.roc_curve_graph(X_train, y_train, X_val, y_val)

In [ ]:
%%time

import lightgbm
from lightgbm import LGBMClassifier
print(lightgbm.__version__)
lgb_params = {
    'metric': 'auc',
    'n_estimators': 10000,
    'objective': 'binary',
}

lgb_model = sk_helper(model=LGBMClassifier, seed=SEED, params=lgb_params)
lgb_model.roc_curve_graph(X_train, y_train, X_val, y_val)

In [ ]:
tree_features = tree_model.feature_importances(X_train, y_train)
rf_features = rf_model.feature_importances(X_train, y_train)
lgb_features = lgb_model.feature_importances(X_train, y_train)

In [ ]:
cols = X.columns.values
feature_df = pd.DataFrame({'features': cols, 
                          'Decision Tree': tree_features, 
                          'RandomForest': rf_features, 
                          'LightGBM': lgb_features})

feature_df

In [ ]:
import numpy as np
from datetime import datetime

version = datetime.now().strftime("%d-%m-%Y %H-%M-%S")

def final_submission(model, data, version):
    final_preds = model.predict(data)
    binarizer = np.vectorize(lambda x: 1 if x >= .5 else 0)
    prediction_binarized = binarizer(final_preds)
    submission = pd.concat([sample_submission,pd.DataFrame(prediction_binarized)], axis=1).drop(columns=['Survived'])
    submission.columns = ['PassengerId', 'Survived']
    submission.to_csv('Sklearn of Submit Date {} Submission.csv'.format(version), index=False)
    
final_submission(lgb_model, test_data, version)

In [ ]:
# !pip uninstall scikit-learn -y
!pip install pycaret==2.2.3

In [ ]:
# check version
from pycaret.utils import version
import sklearn
print("pycaret version:", version())
print("sklearn version:", sklearn.__version__)

In [ ]:
all_df_pycaret = pd.concat([X, y], axis=1)
all_df_pycaret['Survived'] = all_df_pycaret['Survived'].astype('int64')
all_df_pycaret.info()

In [ ]:
from pycaret.classification import *

setup(data = all_df_pycaret, 
      target = 'Survived', 
      fold = 3,
      silent = True
     )

set_config('seed', 123)

In [ ]:
%%time

best_model = compare_models(sort = 'Accuracy', n_select = 3)

In [ ]:
%%time
gbc_model = create_model('gbc')

In [ ]:
%%time
tuned_gbc = tune_model(gbc_model, n_iter = 50)

In [ ]:
plot_model(tuned_gbc, plot = 'feature_all')

In [ ]:
predictions = predict_model(tuned_gbc, data = test_data)
predictions.info()

In [ ]:
submission = pd.read_csv(BASE_DIR + 'sample_submission.csv')
submission['Survived'] = predictions['Label']
submission.to_csv('PyCaret Submission.csv', index=False)
submission.head()